In [1]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
path = "drive/My Drive/logistic.txt"

In [4]:
!pip install PyDrive

     |████████████████████████████████| 993kB 3.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from pydrive.auth import GoogleAuth
auth.authenticate_user()
gauth = GoogleAuth()


In [0]:
gauth.credentials = GoogleCredentials.get_application_default()
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
download = drive.CreateFile({'id': '1-Oduujt-fDn-Gkuc29uXSCrf1ejqTp6b'}) #visitors pool for 04_12_18 with choosen features

In [0]:
download.GetContentFile('DOWNLOAD.json')

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
df=pd.read_json('DOWNLOAD.json', lines=True)

In [0]:
download_05 = drive.CreateFile({'id': '1RzwqLZb62wYMnqsRT9o_QA3yd949-g4W'}) #visitors pool for next day 05_12_18 with only browserId and unixTimestamp columns

In [0]:
download_05.GetContentFile('DOWNLOAD.json')

In [0]:
visits=pd.read_json('DOWNLOAD.json', lines=True)

In [18]:
df.describe()

,adblockerStatus,hour,isNewVisit,parsedReferrerId
count,1.088111e+06,1088111.0,1.088106e+06,1.088111e+06
mean,-8.489943e-03,0.0,9.885452e-01,8.203375e+00
std,3.980446e-01,0.0,1.064122e-01,5.530869e+00
min,-2.000000e+00,0.0,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.0,1.000000e+00,2.000000e+00
50%,0.000000e+00,0.0,1.000000e+00,1.200000e+01
75%,0.000000e+00,0.0,1.000000e+00,1.300000e+01
max,1.000000e+00,0.0,1.000000e+00,1.300000e+01


In [19]:
df.columns

Index(['adblockerStatus', 'browser', 'browserId', 'city', 'content',
       'contentAuthor', 'contentSection', 'hour', 'isFirstVisit', 'isNewVisit',
       'parsedReferrerId', 'region', 'unixTimestamp'],
      dtype='object')

In [0]:
visitors=visits['browserId']

In [21]:
visitors #the list of visitors on the next day, First-time visitors are excluded

0          s8Chgmh2CKPQ2vPi4DHIxQ==
1          1iJqSIJH2Fu8gHRjI86u0g==
2          HN58tOi94UO07TzHFhwrvg==
3          qzujOQagBTP/PO58Bc9rww==
4          TBqIQQIkAQDbVUvrcXob3w==
5          pww+XQPTxxh3IzUDzV3dPg==
6          uGe6sUJnjNI8Ho3ojwbPyg==
7          dpRAXTZlRKl2FWkTlxmuFg==
8          bMJ6o4OTkWCLVR1WYT+JFw==
9          mQZ+DlGvofG6ggkG3/lj+A==
10         spvtE/7r6bD/thFS6fxEjg==
11         CH6NU2XFDRjYMnT7qcel2A==
12         J5XPyLxfaQKj2OR0igVmSA==
13         KXe6D+ez9Q9gcq9Tp+cJhQ==
14         6VuZdziQ31Rdvm3sezBygg==
15         L//U0x5Ng2KzX5c+TDdKtg==
16         /TaT49bGluKr7UCE3fCV6Q==
17         Ce1WJdff08xjoZiPSHUzHg==
18         TaFbO/wXdTq+xPXZ1kHOIg==
19         7b1+c9VnUu0XSTBlV11RAQ==
20         hK1klzf5WYmmCk37RGUj3w==
21         yRZ/VbjTltlOsbdbJf157w==
22         WsjIJkStMklfXlAjJEacKQ==
23         TzzGCUe4qE/UnBj6YZUmcA==
24         QLR5Bll6mmoKp0dRYtkPtg==
25         v/a9EXwIBTQugPj/kPmU0Q==
26         vNROAERoERg3VgJD9juzYw==
27         Bssar/uHLamcNx4Pr

In [0]:
retention = df['browserId'].isin(visitors) #determine visitors who returns next day with 'True' value
df['Ret']=retention

In [23]:
df

,adblockerStatus,browser,browserId,city,content,contentAuthor,contentSection,hour,isFirstVisit,isNewVisit,parsedReferrerId,region,unixTimestamp,Ret
0,0,safari,YZ49jp/DA0M6yYbrPrmd+w==,NaN,"{'id': '', 'keywords': '5y bond,u.s. 2 year tr...",Pt8bcy625lIuWVrQ1wDeuA==,,0,True,1.0,11,NaN,2018-12-04 00:00:00.03 UTC,False
1,0,safari,KXCew/knnnZ3/ELhGLfAnw==,Boston,"{'id': '', 'keywords': 'technology, alphabet c...",Kjz3YeHWfKR1kJ/tjtel3w==,Technology,0,True,1.0,13,Massachusetts,2018-12-04 00:00:00.071 UTC,False
2,0,chrome,X6n0RhlP8SPyXHk/+xrEfw==,Bronx,"{'id': '', 'keywords': 'politics, trade, white...",2k3MrScgYinhO2IcDCvCqg==,Politics,0,True,1.0,12,New York,2018-12-04 00:00:00.095 UTC,False
3,0,chrome,rG+IpRGPdhSe+04rAgwFkA==,Dundalk,"{'id': '', 'keywords': 'politics, robert muell...",yZnt3PG6YAGJR+/6Q3IySg==,Politics,0,True,1.0,12,Maryland,2018-12-04 00:00:00.156 UTC,False
4,1,chrome,MUpACX53Z4So3CoNOX5zWg==,Racine,"{'id': '', 'keywords': 'cramer remix, mad mone...",XIJUwwPcS3C77OuC5aVnOg==,Mad Money,0,True,1.0,13,Wisconsin,2018-12-04 00:00:00.219 UTC,False
5,0,chrome,xG+N2JLRDLb1WnYyI8+JaA==,Kuala Lumpur,"{'id': '', 'keywords': 'oil, asia economy, bre...",ibcBdNZrHCzEgdQ53nR0jw==,Oil,0,True,1.0,13,Kuala Lumpur,2018-12-04 00:00:00.292 UTC,False
6,0,firefox,ICTY9aAGD2UoVx8RoYiHJg==,Reading,"{'id': '', 'keywords': '', 'title': 'JWCx6fc1d...",1B2M2Y8AsgTpgAmY7PhCfg==,Squawk Box Europe,0,True,1.0,12,Reading,2018-12-04 00:00:00.413 UTC,False
7,0,safari,3kIXf284YA2sHGHBnU0WWA==,Brooklyn,"{'id': '', 'keywords': 'airlines, business, am...",UmgjY2D8N6gNBjxRy2+4BA==,Airlines,0,True,1.0,11,New York,2018-12-04 00:00:00.502 UTC,False
8,0,chrome,mESb3diwKDZrwuHtLwCTDg==,Granby,"{'id': '', 'keywords': 'airlines, federal avia...",UmgjY2D8N6gNBjxRy2+4BA==,Airlines,0,True,1.0,0,Colorado,2018-12-04 00:00:00.561 UTC,False
9,0,misc,wsTGXNJjOb6FDUZulLQL7Q==,Everett,"{'id': '', 'keywords': 'politics, robert muell...",yZnt3PG6YAGJR+/6Q3IySg==,Politics,0,True,1.0,2,Washington,2018-12-04 00:00:00.587 UTC,False


In [24]:
sum(df['Ret']) #true values

43219

In [25]:
len(df['Ret'])-sum(df['Ret']) #false values, we see that data is unbalanced

1044892

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import featuretools as ft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

In [27]:
df_majority = df[df['Ret']==False] #balancing our dataset
df_minority = df[df['Ret']==True]
# resample minority class

from sklearn.utils import resample
df_majority_upsampled = resample(df_majority, 
                                 replace=True,     
                                 n_samples=43219, random_state=123)   
df_up = pd.concat([df_minority, df_majority_upsampled])
df_up['Ret'].value_counts() #resampled dataframe

True     43219
False    43219
Name: Ret, dtype: int64

In [0]:
X = df_up['contentAuthor'] #one feature as input
Y = df_up['Ret']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
cvec = CountVectorizer(stop_words='english').fit(X_train) #in this work we will use 3 embedding alghorithms, CountVectorizer is the first
df_train=pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())
df_test=pd.DataFrame(cvec.transform(X_test).todense(), columns=cvec.get_feature_names())

In [103]:
lr = LogisticRegression(penalty='l1') #logistic regression for sparse matrice
lr.fit(df_train, Y_train)
Y_pred=lr.predict(df_test)
print(lr.score(df_test, Y_test))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5981419807186679
[[9123 5087]
 [6376 7939]]
0.609473360970367


In [0]:

dfna=df_up.dropna()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']

X5=dfna['contentAuthor']
X6=dfna['contentSection']

X8=dfna['region']
Y=dfna['Ret']
X2=cvec.fit_transform(X2)
X3=cvec.fit_transform(X3)

X5=cvec.fit_transform(X5)
X6=cvec.fit_transform(X6)

X8=cvec.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X5, X6, X8])
(X_train, X_test) = train_test_split(X)
(Y_train, Y_test) = train_test_split(Y)

In [106]:
lr = LogisticRegression(penalty='l1') 
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(lr.score(X_test, Y_test))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5147741663149008
[[2330 6556]
 [2640 7426]]
0.5311114289801173


In [0]:
from sklearn import tree #decision tree alghorithm
clf = tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')
clf = clf.fit(X_train, Y_train)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
Y_pred=clf.predict(X_test)

In [109]:
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))#not the best score with multiple categorial features

0.5311312790206838
[[   10  8876]
 [   10 10056]]
0.5311641664905979


In [110]:
X = df_up['contentAuthor'] 
Y = df_up['Ret']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train=pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())
df_test=pd.DataFrame(cvec.transform(X_test).todense(), columns=cvec.get_feature_names())
clf = clf.fit(df_train, Y_train)
Y_pred=clf.predict(df_test) #randomforest classifier for contentAuthor feature
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))


0.592182296231376
[[11076  3263]
 [ 8370  5816]]
0.6405991849322613


array([[10989,  3221],
       [ 8374,  5941]])

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X=df_up['contentAuthor']
Y = df_up['Ret']
X=vectorizer.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(lr.score(X_test, Y_test))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))


0.6017879053461875
[[9543 4790]
 [6569 7623]]
0.6141142350761298


In [112]:
!pip install category_encoders
import category_encoders as ce
ce_binary = ce.BinaryEncoder(cols = ['contentAuthor'])

In [0]:
x=df_up['contentAuthor']
y=df_up['Ret']
x=ce_binary.fit_transform(x) #binary encoder as alternate to TF-Idf, Tf and CVEC encoders
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)

In [114]:
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5886415425065732
[[10328  3995]
 [ 7739  6463]]
0.6179957926945878


In [116]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5865030674846625
[[8888 5435]
 [6360 7842]]
0.5906454771409204


In [0]:
x=df_up['contentSection']
y=df_up['Ret']
ce_binary = ce.BinaryEncoder(cols = ['contentSection'])
x=ce_binary.fit_transform(x)

In [118]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5797721297107801
[[11842  2480]
 [ 9507  4696]]
0.6544035674470458


In [119]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5893426818580193
[[10253  4069]
 [ 7645  6558]]
0.6171073680248423


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
X=df_up['contentSection']
X=vectorizer.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)


In [121]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.6112182296231375
[[10370  3947]
 [ 7143  7065]]
0.6415728296403923


In [0]:
x=df_up['contentSection']
y=df_up['Ret']
ce_binary = ce.BinaryEncoder(cols = ['contentSection'])
x=ce_binary.fit_transform(x)

In [123]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5855915863277826
[[11869  2416]
 [ 9405  4835]]
0.6668045786788029


In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB() #naive Bayes Alghorithm


In [125]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.547589833479404
[[ 3564 10721]
 [ 2184 12056]]
0.5293058787373227


In [0]:
dfna=df_up.dropna()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']

X8=dfna['region']
Y=dfna['Ret']

X2=cvec.fit_transform(X2)
X3=cvec.fit_transform(X3)
X4=cvec.fit_transform(X4)
X5=cvec.fit_transform(X5)
X6=cvec.fit_transform(X6)

X8=cvec.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [127]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5805083260297984
[[10189  4041]
 [ 7925  6370]]
0.6118528479492844


In [128]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5441893076248905
[[ 3454 10776]
 [ 2226 12069]]
0.5282994090610637


In [129]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.59011393514461
[[10233  3997]
 [ 7695  6600]]
0.6228177786165896


In [130]:
from sklearn.neighbors import KNeighborsClassifier #knn for multiple features
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(X_train, Y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')

In [131]:
Y_pred=neigh.predict(X_test) 
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5883961437335671
[[9561 4669]
 [7072 7223]]
0.6073831146989572


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF for embedding
vectorizer = TfidfVectorizer()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']
X8=dfna['region']
Y=dfna['Ret']

X2=vectorizer.fit_transform(X2)
X3=vectorizer.fit_transform(X3)
X4=vectorizer.fit_transform(X4)
X5=vectorizer.fit_transform(X5)
X6=vectorizer.fit_transform(X6)
X8=vectorizer.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [133]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5804732690622261
[[11794  2489]
 [ 9478  4764]]
0.656831655866538


In [134]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5474145486415425
[[ 3694 10589]
 [ 2321 11921]]
0.5295868502887605


In [135]:
neigh.fit(X_train, Y_train)
Y_pred=neigh.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5582471516213847
[[6884 7399]
 [5202 9040]]
0.5499117951213578


In [136]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.590920245398773
[[10193  4090]
 [ 7579  6663]]
0.6196410304101181


In [0]:
from sklearn.feature_extraction.text import FeatureHasher #feature hashing
hasher = FeatureHasher(input_type='string')

X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']
X8=dfna['region']
Y=dfna['Ret']

X2=hasher.fit_transform(X2)
X3=hasher.fit_transform(X3)
X4=hasher.fit_transform(X4)
X5=hasher.fit_transform(X5)
X6=hasher.fit_transform(X6)
X8=hasher.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [138]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33) #for all of  the 5 alghorithms
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5845048203330412
[[11868  2456]
 [ 9396  4805]]
0.6617545792590552


In [139]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.548396143733567
[[ 3623 10701]
 [ 2181 12020]]
0.5290260111790854


In [140]:
neigh.fit(X_train, Y_train)
Y_pred=neigh.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5960035056967572
[[8686 5638]
 [5886 8315]]
0.5959291908550133


In [141]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.5949167397020158
[[10299  4025]
 [ 7530  6671]]
0.6236910994764397


In [142]:
X=dfna['hour']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test, Y_pred, normalize=True))
print(confusion_matrix(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))



0.579070990359334
[[10155  4071]
 [ 7936  6363]]
0.6098332374928119


In [0]:
dfna

In [0]:
x=df_up[['adblockerStatus', 'browser', 'browserId', 'city', 'content',
       'contentAuthor', 'contentSection', 'hour', 'isFirstVisit', 'isNewVisit',
       'parsedReferrerId', 'region']]
y=df_up['Ret']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [145]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import log_loss
X_train_hash = X_train
X_test_hash = X_test
for i in range(X_train_hash.shape[1]):
    X_train_hash.iloc[:,i]=X_train_hash.iloc[:,i].astype('str')
for i in range(X_test_hash.shape[1]):
    X_test_hash.iloc[:,i]=X_test_hash.iloc[:,i].astype('str')
h = FeatureHasher(n_features=100,input_type="string")
X_train_hash = h.transform(X_train_hash.values)
X_test_hash = h.transform(X_test_hash.values)
lr.fit(X_train_hash,y_train)
y_pred = lr.predict(X_test_hash)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


0.6107274320771253
[[8835 5434]
 [5670 8586]]
0.6124108416547789


In [151]:
clf.fit(X_train_hash,y_train)
y_pred = clf.predict(X_test_hash)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


0.5751446099912357
[[12887  1382]
 [10737  3519]]
0.7180167312793307


In [147]:
neigh.fit(X_train_hash, y_train)
y_pred=neigh.predict(X_test_hash)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


0.6182646801051709
[[9636 4633]
 [6256 8000]]
0.6332620913480567


In [153]:
X_train_count = X_train
X_test_count = X_test
X_train_count["test"]=0
X_test_count["test"]=1
temp_df = pd.concat([X_train_count,X_test_count],axis=0)
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('category')
X_train_count=temp_df[temp_df["test"]==0].iloc[:,:-1]
X_test_count=temp_df[temp_df["test"]==1].iloc[:,:-1]
for i in range(X_train_count.shape[1]):
    counts = X_train_count.iloc[:,i].value_counts()
    counts = counts.sort_index()
    counts = counts.fillna(0)
    counts += np.random.rand(len(counts))/1000
    X_train_count.iloc[:,i].cat.categories = counts
    X_test_count.iloc[:,i].cat.categories = counts
                       
lr.fit(X_train_count,y_train)
y_pred = lr.predict(X_test_count)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))



0.587239263803681
[[13022  1247]
 [10527  3729]]
0.7493971061093248


In [152]:
clf.fit(X_train_count, y_train)
y_pred = clf.predict(X_test_count)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))

0.6005609114811569
[[12810  1459]
 [ 9935  4321]]
0.7475778546712802


In [149]:
model.fit(X_train_count,y_train)
y_pred = model.predict(X_test_count)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


0.5814548641542506
[[12676  1593]
 [10346  3910]]
0.710521533708886


In [150]:
neigh.fit(X_train_count, y_train)
y_pred=neigh.predict(X_test_count)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


0.6167221735319894
[[10010  4259]
 [ 6674  7582]]
0.6403175407482476


In [154]:
df_majority = df[df['Ret']==False] #balancing our dataset with increased number of samples
df_minority = df[df['Ret']==True]
# resample minority class

from sklearn.utils import resample
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     
                                 n_samples=1044892, random_state=123)   
df_up = pd.concat([df_majority, df_minority_upsampled])
df_up['Ret'].value_counts() #resampled dataframe

True     1044892
False    1044892
Name: Ret, dtype: int64

In [0]:
x=df_up[['adblockerStatus', 'browser', 'browserId', 'city', 'content',
       'contentAuthor', 'contentSection', 'hour', 'isFirstVisit', 'isNewVisit',
       'parsedReferrerId', 'region']]
y=df_up['Ret']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [0]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import log_loss
X_train_hash = X_train
X_test_hash = X_test
for i in range(X_train_hash.shape[1]):
    X_train_hash.iloc[:,i]=X_train_hash.iloc[:,i].astype('str')
for i in range(X_test_hash.shape[1]):
    X_test_hash.iloc[:,i]=X_test_hash.iloc[:,i].astype('str')
h = FeatureHasher(n_features=100,input_type="string")
X_train_hash = h.transform(X_train_hash.values)
X_test_hash = h.transform(X_test_hash.values)
lr.fit(X_train_hash,y_train)
y_pred = lr.predict(X_test_hash)
print(accuracy_score(y_test, y_pred, normalize=True))
print(confusion_matrix(y_test, y_pred))
print(precision_score(y_test, y_pred))


In [0]:
neigh.fit(X_train_hash, y_train)
y_pred=neigh.predict(X_test_hash)
accuracy_score(y_test, y_pred, normalize=True)